## Pretraining using SAT-6 data

#### First, read the files created earlier

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import fiona
from classification import simplenet4 as net

# Read files 
trainX = np.load('/scratch/slums/bl-slums/gt/trainSATX')
trainY = np.load('/scratch/slums/bl-slums/gt/trainSATY')
valX = np.load('/scratch/slums/bl-slums/gt/valSATX')
valY = np.load('/scratch/slums/bl-slums/gt/valSATY')
testX = np.load('/scratch/slums/bl-slums/gt/testSATX')
testY = np.load('/scratch/slums/bl-slums/gt/testSATY')

In [2]:
# Prepare the training task
import sys,os, datetime
from tflearn.data_utils import shuffle
# Get the model
model = net.model


# Load data
trainX = trainX.reshape([-1, 28, 28, 4])
print trainX.shape
valX = valX.reshape([-1, 28, 28, 4])
print valX.shape

# Shuffle data
trainX, trainY = shuffle(trainX, trainY)


model.fit(trainX, trainY, n_epoch=32,validation_set=(valX, valY), show_metric=True, run_id="deep_nn2", batch_size = 256,snapshot_step=100000)
model.save("/scratch/slums/bl-slums/model/final-model.tflearn")

Training Step: 32415  | total loss: 0.04493 | time: 494.787s
| Adam | epoch: 032 | loss: 0.04493 - acc: 0.9827 -- iter: 259072/259197
Training Step: 32416  | total loss: 1.47061 | time: 519.684s
| Adam | epoch: 032 | loss: 1.47061 - acc: 0.9071 | val_loss: 0.02634 - val_acc: 0.9910 -- iter: 259197/259197
--
INFO:tensorflow:/scratch/slums/bl-slums/model/model--32416 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/scratch/slums/bl-slums/model/best-model-9910 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/scratch/slums/bl-slums/model/model--32416 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:/scratch/slums/bl-slums/model/final-model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
print len(testX)

In [3]:
#predict for the model
testX = np.asarray(testX)
predicted_all = []
actual_all = []
correct = np.zeros((1,6))
total = np.zeros((1,6))
testX = np.asarray(testX)
print testX.shape
for i in range(testX.shape[0]):
    current = testX[i]
    current = current.reshape(-1,28,28,4)
    result = model.predict(current)[0] # Predict
    #print result
    prediction = np.argmax(result) # The index represents the number predicted in this case
    actual = np.argmax(testY[i])
    #print result, testY[i]
    if actual == prediction:
        correct[0,actual]+=1
    total[0,actual]+=1
    #print("Prediction: {}, Actual: {}".format(prediction,actual))
    predicted_all.append(prediction+1)
    actual_all.append(actual+1)
# Generate accuracy stats
accuracies = np.zeros((1,6))
for k in range(6):
     accuracies[0,k] = correct[0,k]/total[0,k]
print accuracies
print np.sum(correct)/np.sum(total)

(81000, 28, 28, 4)
[[ 0.97065159  0.98649752  0.99703913  0.97642109  0.9821256   1.        ]]
0.990950617284


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(actual_all, predicted_all)